## Multivariate_LSTM

- a total of 334 months [0-33]
    - Train set: [0-32] months
    - Test set: 33 month
- Train on the (shop, item) tuples with median >0 and appear in test set
- Create the time lag features
    - shop time lag: shop_sum_month
    - item time lag: item_sum_month
- ttl training feature shape: (1175113, 22)
- input feature shape (1019418, 1, 21)
    - (# num of samples, # num of time steps, # num of features)
    
### Final output
- Training mse: 0.0237
- Training mae: 0.0562
- Validation mse: 0.0410
- Validation mae: 0.0281


In [129]:
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
%matplotlib inline
from xgboost import XGBRegressor
from xgboost import plot_importance
from itertools import product
import time
import pickle
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from datetime import datetime, timedelta
import matplotlib.dates as mdates
import datetime as dt
import re
import os
import sqlite3
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import mean_squared_error, classification_report
from sklearn.svm import SVR
from sklearn import preprocessing
# from statsmodels.sandbox.regression.predstd import wls_prediction_std
# import statsmodels.api as sm
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from numpy import array
from numpy import hstack
# from statsmodels.graphics.gofplots import qqplot
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
import keras

In [2]:
# merge train with item data
def file_loc(path):
    filename = 'data/'
    file_dir = os.path.join(filename,path)
    return file_dir

train_file = file_loc('sales_train.csv')
item_file = file_loc('items.csv')
shops_file = file_loc('shops.csv')
item_cate_file = file_loc('item_categories.csv')
test = file_loc('test.csv')

df_train = pd.read_csv(train_file)
df_item = pd.read_csv(item_file)
df_shops = pd.read_csv(shops_file)
df_item_cate = pd.read_csv(item_cate_file)
df_test = pd.read_csv(test)

print('training data shape: ', df_train.shape)
print(df_train.head())

training data shape:  (2935849, 6)
         date  date_block_num  shop_id  item_id  item_price  item_cnt_day
0  02.01.2013               0       59    22154      999.00           1.0
1  03.01.2013               0       25     2552      899.00           1.0
2  05.01.2013               0       25     2552      899.00          -1.0
3  06.01.2013               0       25     2554     1709.05           1.0
4  15.01.2013               0       25     2555     1099.00           1.0


In [3]:
df_matrix = pd.read_csv('preprocess_matrix.csv', low_memory=False)
df_matrix.head()

,date_block_num,shop_id,item_id,date,item_price,item_cnt_day,item_name,item_category_id,month,shop_item_sum_month,...,date_cat_avg_item_cnt,item_avg_item_price,date_item_avg_item_price,year,shop_item_month_year,item_month_year,shop_day_month,item_day_month,shop_item_month,shop_item_month_un
0,0,2,32,NaN,0.0,0.0,0,0.0,0.0,0.0,...,10.812327,170.302401,336.575462,2013.0,1.0,13.0,65477,728,1007,65477
1,0,2,33,2013-01-05,499.0,1.0,1+1 (BD),37.0,1.0,1.0,...,74.580922,108.304828,391.372549,2013.0,4.0,13.0,1325,91,1007,1325
2,0,2,42,NaN,0.0,0.0,0,0.0,0.0,0.0,...,10.812327,19.731225,15.090909,2013.0,1.0,12.0,65477,1175,1007,65477
3,0,2,45,NaN,0.0,0.0,0,0.0,0.0,0.0,...,10.812327,29.558935,61.558824,2013.0,1.0,13.0,65477,1185,1007,65477
4,0,2,51,NaN,0.0,0.0,0,0.0,0.0,0.0,...,10.812327,34.104661,60.567568,2013.0,1.0,13.0,65477,1240,1007,65477


In [69]:
df_matrix1 = df_matrix.copy()
col_lst = ['date_block_num', 'item_id','shop_id',  'item_avg_item_price','shop_item_sum_month',
           'shop_item_month_year', 'item_month_year', 'shop_day_month', 'item_day_month', 'shop_item_month_un',
            'shop_item_month']
df_matrix2 = df_matrix1[col_lst]
df_matrix2.shape

(3570291, 11)

In [95]:
# loc shop item tup list no sales on month 32
## the tup (shop, item ) only in train not in test
shop_item_train = set(list(zip(df_train['shop_id'], df_train['item_id'])))
shop_item_test = set(list(zip(df_test['shop_id'], df_test['item_id'])))
test_not_train_set = shop_item_test-shop_item_train
# loc shop item tup list no sales on month 33
df_pro = df_matrix4.loc[df_matrix4['date_block_num']<32]
df_pro1 = df_matrix4.loc[df_matrix4['date_block_num']==32]
# df_pro1 = df_pro.groupby(['shop_id','item_id','date_block_num'])['item_cnt_day'].sum().reset_index()
df_pro2 = df_pro.groupby(['shop_id','item_id'])['shop_item_sum_month'].median().reset_index()
df_pro3 = df_pro2.loc[(df_pro2['shop_item_sum_month']==0)]
shop_item_3 = set(list(zip(df_pro3['shop_id'], df_pro3['item_id'])))
shop_item_1 = set(list(zip(df_pro1['shop_id'], df_pro1['item_id'])))
shop_item_2 = set(list(zip(df_pro['shop_id'], df_pro['item_id'])))

for shop, item in (shop_item_2-shop_item_1-shop_item_3-test_not_train_set):
    
    df_sam = df_matrix4.loc[(df_matrix4['shop_id']==shop) & (df_matrix4['item_id']==item) & (df_matrix4['date_block_num']<32)]
    
    df_date_less = df_sam.iloc[-1,:]
    df_date_less1 = df_date_less.to_frame().T

    df_date_less1['date_block_num'] = 32
#     print(df_matrix2.shape)
    df_matrix4 = pd.concat([df_matrix4,df_date_less1])
#     print(df_matrix2.shape)
#     print(type(df_date_proc33))
#     break
# shop_item_33 = set(list(zip(df_pro1['shop_id'], df_pro1['item_id'])))
# len(shop_item_33)
df_matrix4.shape


(3301764, 22)

In [70]:
## the tup (shop, item ) only in train not in test
shop_item_train = set(list(zip(df_train['shop_id'], df_train['item_id'])))
shop_item_test = set(list(zip(df_test['shop_id'], df_test['item_id'])))
test_not_train_set = shop_item_test-shop_item_train
# loc shop item tup list no sales on month 33
df_pro = df_matrix1.loc[df_matrix1['date_block_num']<33]
df_pro1 = df_matrix1.loc[df_matrix1['date_block_num']==33]
df_pro2 = df_pro.groupby(['shop_id','item_id'])['shop_item_sum_month'].median().reset_index()
df_pro3 = df_pro2.loc[(df_pro2['shop_item_sum_month']==0)]
shop_item_3 = set(list(zip(df_pro3['shop_id'], df_pro3['item_id'])))
shop_item_1 = set(list(zip(df_pro1['shop_id'], df_pro1['item_id'])))
shop_item_2 = set(list(zip(df_pro['shop_id'], df_pro['item_id'])))

print('item_shop_mod shape:',len(shop_item_1-shop_item_2-shop_item_3-test_not_train_set))

for shop, item in (shop_item_2-shop_item_1-shop_item_3-test_not_train_set):
    
    df_sam = df_matrix2.loc[(df_matrix2['shop_id']==shop) & (df_matrix2['item_id']==item) & (df_matrix2['date_block_num']<33)]
    
    df_date_less = df_sam.iloc[-1,:]
    df_date_less1 = df_date_less.to_frame().T

    df_date_less1['date_block_num'] = 33
    df_matrix2 = pd.concat([df_matrix2,df_date_less1])

df_matrix2.shape

item_shop_mod shape: 2869


(3572338, 11)

In [111]:
def lstm(x_train, Y_train, X_test, y_test, n_step, n_features, input_LSTM):
    model = Sequential()
    model.add(LSTM(input_LSTM, activation='relu', input_shape=(n_step, n_features)))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse', metrics=['mse','mae'])
    # fit model
    history =model.fit(x_train, Y_train, epochs=80, verbose=2, validation_data=(X_test, y_test),batch_size = 120)
    return history, model

def lstm_2layer(x_train, Y_train, X_test, y_test, n_step, n_features, input_LSTM):
    model = Sequential()
    model.add(LSTM(input_LSTM, activation='relu', input_shape=(n_step, n_features), return_sequences=True))
    model.add(LSTM(30, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse', metrics=['mse','mae'])
    # fit model
    history =model.fit(x_train, Y_train, epochs=80, verbose=2, validation_data=(X_test, y_test),batch_size = 120)
    return history, model

In [96]:
df_matrix4['item_sum_month_-1'] = df_matrix4.groupby(['item_id'])['shop_item_sum_month'].shift(-1)
df_matrix4['item_sum_month_1'] = df_matrix4.groupby(['item_id'])['shop_item_sum_month'].shift(1)
df_matrix4['item_sum_month_2'] = df_matrix4.groupby(['item_id'])['shop_item_sum_month'].shift(2)
df_matrix4['item_sum_month_3'] = df_matrix4.groupby(['item_id'])['shop_item_sum_month'].shift(3)
df_matrix4['item_sum_month_6'] = df_matrix4.groupby(['item_id'])['shop_item_sum_month'].shift(6)
df_matrix4['shop_sum_month_-1'] = df_matrix4.groupby(['shop_id'])['shop_item_sum_month'].shift(-1)
df_matrix4['shop_sum_month_1'] = df_matrix4.groupby(['shop_id'])['shop_item_sum_month'].shift(1)
df_matrix4['shop_sum_month_2'] = df_matrix4.groupby(['shop_id'])['shop_item_sum_month'].shift(2)
df_matrix4['shop_sum_month_3'] = df_matrix4.groupby(['shop_id'])['shop_item_sum_month'].shift(3)
df_matrix4['shop_sum_month_6'] = df_matrix4.groupby(['shop_id'])['shop_item_sum_month'].shift(6)
df_matrix4["item_sum_month_-1"].fillna(0, inplace=True)
df_matrix4["item_sum_month_1"].fillna(0, inplace=True)
df_matrix4["item_sum_month_2"].fillna(0, inplace=True)
df_matrix4["item_sum_month_3"].fillna(0, inplace=True)
df_matrix4["item_sum_month_6"].fillna(0, inplace=True)
df_matrix4["shop_sum_month_-1"].fillna(0, inplace=True)
df_matrix4["shop_sum_month_1"].fillna(0, inplace=True)
df_matrix4["shop_sum_month_2"].fillna(0, inplace=True)
df_matrix4["shop_sum_month_3"].fillna(0, inplace=True)
df_matrix4["shop_sum_month_6"].fillna(0, inplace=True)


In [103]:
# loc shop item tup list sales median >0 intersec with
## the tup (shop, item ) only in train not in test
shop_item_test = set(list(zip(df_test['shop_id'], df_test['item_id'])))
test_not_train_set = shop_item_test-shop_item_train
df_pro = df_matrix4.loc[df_matrix4['date_block_num']<=33]
df_pro2 = df_pro.groupby(['shop_id','item_id'])['shop_item_sum_month'].median().reset_index()
df_pro3 = df_pro2.loc[(df_pro2['shop_item_sum_month']>0)]
shop_item_1 = set(list(zip(df_pro3['shop_id'], df_pro3['item_id'])))
z = shop_item_1.intersection(shop_item_test) 

matrix_lst = []
column_names = ['date_block_num', 'item_id', 'shop_id', 'item_avg_item_price', 'shop_item_sum_month',
                'shop_item_month_year', 'item_month_year', 'shop_day_month', 'item_day_month', 
                'shop_item_month_un', 'shop_item_month', 'ID', 'item_sum_month_-1', 'item_sum_month_1',
                'item_sum_month_2', 'item_sum_month_3', 'item_sum_month_6', 'shop_sum_month_-1', 
                'shop_sum_month_1', 'shop_sum_month_2', 'shop_sum_month_3', 'shop_sum_month_6']

df = pd.DataFrame(columns = column_names)

for shop, item in z:
    
    df_matrix5 = df_matrix4.loc[(df_matrix4['shop_id']==shop) & (df_matrix4['item_id']==item)]
    df = pd.concat([df,df_matrix5])
    
print('ttl feature df shape:',df.shape)
    
    

ttl feature df shape: (1175113, 22)


In [107]:


column_names = ['date_block_num', 'item_id', 'shop_id', 'item_avg_item_price', 'shop_item_sum_month',
                'shop_item_month_year', 'item_month_year', 'shop_day_month', 'item_day_month', 
                'shop_item_month_un', 'shop_item_month', 'ID',  'item_sum_month_1',
                'item_sum_month_2', 'item_sum_month_3', 'item_sum_month_6', 'shop_sum_month_-1', 
                'shop_sum_month_1', 'shop_sum_month_2', 'shop_sum_month_3', 'shop_sum_month_6']
label_col = ['date_block_num','item_sum_month_-1']


feature_train = df[column_names]
label_train = df[label_col]
X_train = feature_train[feature_train.date_block_num < 32]
X_train.sort_values(by=['date_block_num', 'shop_id','item_id'], inplace=True)
label_train.sort_index(inplace=True)
y_train = label_train.loc[label_train.date_block_num < 32, 'item_sum_month_-1'].to_frame()
X_train_idx = X_train.index.values
y_train = y_train.reindex(X_train_idx)
Y_train = y_train.values
X_valid = feature_train[feature_train.date_block_num == 32]
X_valid.sort_values(by=['date_block_num', 'shop_id','item_id'], inplace=True)
y_valid = label_train.loc[label_train.date_block_num == 32, 'item_sum_month_-1'].to_frame()
X_valid_idx = X_valid.index.values
y_valid = y_valid.reindex(X_valid_idx)
Y_valid = y_valid.values
X_valid = X_valid.to_numpy()

Y_train = Y_train.reshape(-1,)
Y_valid = Y_valid.reshape(-1,)


X_train_3d, y_train = split_sequences(X_train.to_numpy(), 1, Y_train)
X_valid_3d, y_valid = split_sequences(X_valid, 1, Y_valid)
X_train_3d.shape
    

/Users/denisechen/Desktop/Denise/Python/Blog_Post/venv/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/Users/denisechen/Desktop/Denise/Python/Blog_Post/venv/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/Users/denisechen/Desktop/Denise/Python/Blog_Post/venv/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s

(1019418, 1, 21)

In [109]:
## Run
n_sample = 1
n_step = X_train_3d.shape[0]
n_features = X_train_3d.shape[-1]
x_train = X_train_3d.reshape(n_step, -1,n_features)
x_val = X_valid_3d.reshape(X_valid_3d.shape[0], -1,X_valid_3d.shape[-1])

x_train.shape
# (108, 1,  11)

(1019418, 1, 21)

In [112]:
history_90, model_90 = lstm(X_train_3d, y_train, X_valid_3d, y_valid, X_train_3d.shape[1], X_train_3d.shape[-1], 90)
history_90_2, model_90_2 = lstm_2layer(X_train_3d, y_train, X_valid_3d, y_valid, X_train_3d.shape[1], X_train_3d.shape[-1], 90)


Epoch 1/80
8496/8496 - 11s - loss: 30328.9180 - mse: 30328.9180 - mae: 34.0180 - val_loss: 208.1668 - val_mse: 208.1668 - val_mae: 10.9920
Epoch 2/80
8496/8496 - 11s - loss: 108.5269 - mse: 108.5269 - mae: 5.5933 - val_loss: 69.8824 - val_mse: 69.8824 - val_mae: 6.7798
Epoch 3/80
8496/8496 - 10s - loss: 80.4687 - mse: 80.4687 - mae: 4.0190 - val_loss: 4.6250 - val_mse: 4.6250 - val_mae: 1.5051
Epoch 4/80
8496/8496 - 10s - loss: 57.9767 - mse: 57.9767 - mae: 3.7560 - val_loss: 4.8789 - val_mse: 4.8789 - val_mae: 1.6644
Epoch 5/80
8496/8496 - 11s - loss: 49.1582 - mse: 49.1582 - mae: 2.9421 - val_loss: 700.3908 - val_mse: 700.3908 - val_mae: 23.3429
Epoch 6/80
8496/8496 - 14s - loss: 41.0030 - mse: 41.0030 - mae: 2.5494 - val_loss: 849.5751 - val_mse: 849.5751 - val_mae: 20.6876
Epoch 7/80
8496/8496 - 14s - loss: 40.7169 - mse: 40.7169 - mae: 2.1325 - val_loss: 1.3141 - val_mse: 1.3141 - val_mae: 0.8671
Epoch 8/80
8496/8496 - 11s - loss: 25.7192 - mse: 25.7192 - mae: 1.9874 - val_loss: 0

Epoch 66/80
8496/8496 - 10s - loss: 0.0281 - mse: 0.0281 - mae: 0.0650 - val_loss: 0.0410 - val_mse: 0.0410 - val_mae: 0.0385
Epoch 67/80
8496/8496 - 10s - loss: 0.0309 - mse: 0.0309 - mae: 0.0611 - val_loss: 0.0438 - val_mse: 0.0438 - val_mae: 0.0541
Epoch 68/80
8496/8496 - 11s - loss: 0.0292 - mse: 0.0292 - mae: 0.0662 - val_loss: 0.0413 - val_mse: 0.0413 - val_mae: 0.0306
Epoch 69/80
8496/8496 - 11s - loss: 0.0286 - mse: 0.0286 - mae: 0.0628 - val_loss: 0.0409 - val_mse: 0.0409 - val_mae: 0.0311
Epoch 70/80
8496/8496 - 10s - loss: 0.0310 - mse: 0.0310 - mae: 0.0658 - val_loss: 0.5543 - val_mse: 0.5543 - val_mae: 0.5675
Epoch 71/80
8496/8496 - 11s - loss: 0.0246 - mse: 0.0246 - mae: 0.0593 - val_loss: 0.0860 - val_mse: 0.0860 - val_mae: 0.1670
Epoch 72/80
8496/8496 - 10s - loss: 0.0276 - mse: 0.0276 - mae: 0.0593 - val_loss: 0.1299 - val_mse: 0.1299 - val_mae: 0.2634
Epoch 73/80
8496/8496 - 10s - loss: 0.0256 - mse: 0.0256 - mae: 0.0640 - val_loss: 0.0505 - val_mse: 0.0505 - val_mae:

8496/8496 - 17s - loss: 0.0389 - mse: 0.0389 - mae: 0.0891 - val_loss: 0.0410 - val_mse: 0.0410 - val_mae: 0.0301
Epoch 52/80
8496/8496 - 18s - loss: 0.0375 - mse: 0.0375 - mae: 0.0812 - val_loss: 0.1543 - val_mse: 0.1543 - val_mae: 0.2405
Epoch 53/80
8496/8496 - 18s - loss: 0.0393 - mse: 0.0393 - mae: 0.0863 - val_loss: 0.0424 - val_mse: 0.0424 - val_mae: 0.0473
Epoch 54/80
8496/8496 - 18s - loss: 0.0364 - mse: 0.0364 - mae: 0.0834 - val_loss: 0.0663 - val_mse: 0.0663 - val_mae: 0.1419
Epoch 55/80
8496/8496 - 18s - loss: 0.0379 - mse: 0.0379 - mae: 0.0745 - val_loss: 0.0409 - val_mse: 0.0409 - val_mae: 0.0268
Epoch 56/80
8496/8496 - 17s - loss: 0.0473 - mse: 0.0473 - mae: 0.0742 - val_loss: 0.0421 - val_mse: 0.0421 - val_mae: 0.0434
Epoch 57/80
8496/8496 - 17s - loss: 0.0379 - mse: 0.0379 - mae: 0.0682 - val_loss: 0.0726 - val_mse: 0.0726 - val_mae: 0.1606
Epoch 58/80
8496/8496 - 17s - loss: 0.0342 - mse: 0.0342 - mae: 0.0638 - val_loss: 0.0410 - val_mse: 0.0410 - val_mae: 0.0281
Epoc

In [130]:
X_pred = feature_train[feature_train.date_block_num == 33]
X_pred.sort_values(by=['date_block_num', 'shop_id','item_id'], inplace=True)
# x_pred = X_pred.to_numpy()
y_pred = label_train.loc[label_train.date_block_num == 33, 'item_sum_month_-1'].to_frame()
X_pred_idx = X_pred.index.values
y_pred = y_pred.reindex(X_pred_idx)
Y_pred = y_pred.values
X_train_3d, y_train = split_sequences(X_pred.to_numpy(), 1, Y_pred)
# df_pred = pd.DataFrame(model_90_2.predict(X_train_3d), columns=['item_cnt_month'])
df_pred = X_pred[['shop_id','item_id']].copy()
df_pred['item_cnt_month'] = model_90_2.predict(X_train_3d)
# df_pred['item_id'] = X_pred['item_id'].columns.tolist()
df_pred['item_cnt_month'] = df_pred['item_cnt_month'].apply(np.int64)
df_pred.head()



/Users/denisechen/Desktop/Denise/Python/Blog_Post/venv/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,shop_id,item_id,item_cnt_month
963,2.0,482.0,0
2478246,2.0,486.0,2
2478247,2.0,486.0,2
2478248,2.0,486.0,0
2248224,2.0,792.0,0


In [139]:
df_pred1 = pd.merge(df_test, df_pred, on=['shop_id','item_id'], how='left')
df_pred1.fillna(0, inplace=True)
df_pred1.drop_duplicates(subset=['shop_id','item_id'], inplace=True)
df_pred1.head()

,ID,shop_id,item_id,item_cnt_month
0,0,5,5037,0.0
1,1,5,5320,0.0
2,2,5,5233,0.0
3,3,5,5232,0.0
4,4,5,5268,0.0


In [140]:
df_pred1.describe()

,ID,shop_id,item_id,item_cnt_month
count,214200.000000,214200.000000,214200.000000,214200.000000
mean,107099.500000,31.642857,11019.398627,0.158441
std,61834.358168,17.561933,6252.644590,1.084183
min,0.000000,2.000000,30.000000,0.000000
25%,53549.750000,16.000000,5381.500000,0.000000
50%,107099.500000,34.500000,11203.000000,0.000000
75%,160649.250000,47.000000,16071.500000,0.000000
max,214199.000000,59.000000,22167.000000,20.000000


In [141]:
submission2 = df_pred1[['ID', 'item_cnt_month']]
submission2.to_csv('xgb_submission_44.csv', index=False)
